# Entries

1. Historical statistics of direct confrontation (team 1 vs team 2)
2. Statistics of *N* previous matches for each team
3. Home player or away player
4. Current points on championship
5. Match day matters?

In [1]:
import sys
sys.path.append("..")

In [2]:
from pprint import pprint
from pymongo import MongoClient
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from db.database import DAO

In [3]:
dao = DAO()
matches_collection = dao.matches()

In [4]:
# Sample match for testing
sample = matches_collection.find_one()
pprint(sample)

{'_id': ObjectId('5ac952926b5b9e2af46c59ef'),
 'arbiter': 'Leonardo Gaciba da Silva',
 'away_team': 'Santos',
 'cards': [{'player': 'Alex Sandro', 'team': 'SAN', 'type': 'yellow'},
           {'player': 'Zé Eduardo', 'team': 'SAN', 'type': 'yellow'},
           {'player': 'Herrera', 'team': 'BOT', 'type': 'yellow'},
           {'player': 'Wesley', 'team': 'SAN', 'type': 'yellow'},
           {'player': 'Alex Sandro', 'team': 'SAN', 'type': 'red'}],
 'coaches': {'away_team': 'Dorival Júnior', 'home_team': 'Joel Santana'},
 'home_team': 'Botafogo',
 'location': {'city': 'Brasil, Rio de Janeiro, RJ', 'stadium': 'Engenhão'},
 'players': {'away_team': [{'name': 'Felipe Garcia', 'position': 'GOL'},
                           {'name': 'Bruno Aguiar', 'position': 'ZAD'},
                           {'name': 'Durval', 'position': 'ZAE'},
                           {'name': 'Maranhão', 'position': 'LAD'},
                           {'name': 'Alex Sandro', 'position': 'LAE'},
                     

## Match Vectorizer
Takes a match in the above format and extracts the most important characteristics in the following order:

<ol>
- Current match statistics
<li> Home team score </li>
<li> Away team score </li>

- Statistics of all confronts of the teams
<li> Goals by home team </li>
<li> Goals by away team </li>
<li> Number of victories of home team </li>
<li> Number o victories of away team </li>
<li> Number of draws </li>

- Statistics of N previous games of each team
<li> Balance of N last games of home team </li>
<li> Balance of N last games of away team </li>
</ol>

In [53]:
# Default match feature extractor. ht is home_team and at is away_team
N = 4
labels = ['ht_score', 'at_score', 'goals_ht', 'goals_at', 'wins_ht', 'wins_at', 'draws', 
          *(['sg_ht']*4), *(['sg_at']*4)]
def vectorize_match(dao, match, N):
    N = 4
    stats = match['statistics']
    score = match['score']
    home_team_last_sg = dao.get_last_N_balances(match['home_team'], N)
    away_team_last_sg = dao.get_last_N_balances(match['away_team'], N)
    return [
            score['home_team'],
            score['away_team'], 
            stats['goals_home_team'], 
            stats['goals_away_team'],
            stats['wins_home_team'], 
            stats['wins_away_team'], 
            stats['previous_draws'], 
            *home_team_last_sg, 
            *away_team_last_sg
           ]

In [55]:
#Vectorize matches from 2016
matches = matches_collection.find({'year':2016})
data = [vectorize_match(dao, match, N) for match in matches]
data = pd.DataFrame(data=data, columns=labels)
data.head()

,ht_score,at_score,goals_ht,goals_at,wins_ht,wins_at,draws,sg_ht,sg_ht,sg_ht,sg_ht,sg_at,sg_at,sg_at,sg_at
0,4,0,50,39,15,8,13,-1,1,1,0,-2,-1,2,0
1,1,0,48,35,16,9,8,-1,0,2,-2,-1,0,-2,-1
2,1,0,92,80,25,19,17,2,-2,0,0,1,1,-2,2
3,1,0,48,62,10,17,12,0,-1,-1,0,0,1,1,1
4,0,1,72,97,21,26,13,-1,1,2,2,1,0,1,0


In [49]:
# Splitting X and Y
Y = data.iloc[:,:2]
X = data.iloc[:,2:]
Y.head() # Y.values to extract the matrix from dataframe

,ht_score,at_score
0,4,0
1,1,0
2,1,0
3,1,0
4,0,1


In [8]:
# TODO: Feature Scaling


In [10]:
# TODO: Train test split


In [56]:
## TESTING ##
#pego todas as partidas
all_matchs = matches_collection.find()

#escolho 3 partidas aleatórias para fazer o teste
array_Match = vectorize_match(dao, all_matchs[1], 4)
match1= array_Match[2:15]
result1= array_Match[0:2]

array_Match = vectorize_match(dao, all_matchs[2], 4)
match2= array_Match[2:15]
result2= array_Match[0:2]

array_Match = vectorize_match(dao, all_matchs[3], 4)
match3= array_Match[2:15]
result3= array_Match[0:2]

#transformo em uma mariz
inputX = np.array([match1,match2,match3], dtype=np.int)
outputY = np.array([result1,result2,result3], dtype=np.int)

pprint(inputX)
pprint(outputY)


array([[ 3,  5,  1,  2,  1,  2, -1,  2,  0,  1, -1, -1, -1],
       [47, 31, 14,  4,  8, -1,  1,  1,  0,  1,  2, -1,  0],
       [51, 72, 14, 18, 14, -1,  0,  2, -2,  1,  0,  1,  0]])
array([[0, 0],
       [1, 0],
       [1, 1]])


## Building the Perceptron ANN

#TODO: Describe inputs and outputs

In [57]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

/home/dcandrade/.anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [58]:
#Creating the neural network
model = Sequential()
act = 'relu'
model.add(Dense(2, input_dim=13, activation=act))
model.add(Dense(2, activation=act))

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])

In [59]:
#training
model.fit(inputX, outputY, epochs=150, batch_size=13)
scores = model.evaluate(inputX, outputY)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Epoch 1/150
3/3 [==============================] - 0s 27ms/step - loss: 125.6728 - acc: 0.0000e+00
Epoch 2/150
3/3 [==============================] - 0s 1ms/step - loss: 0.5000 - acc: 1.0000
Epoch 3/150
3/3 [==============================] - 0s 1ms/step - loss: 0.5000 - acc: 1.0000
Epoch 4/150
3/3 [==============================] - 0s 1ms/step - loss: 0.5000 - acc: 1.0000
Epoch 5/150
3/3 [==============================] - 0s 724us/step - loss: 0.5000 - acc: 1.0000
Epoch 6/150
3/3 [==============================] - 0s 894us/step - loss: 0.5000 - acc: 1.0000
Epoch 7/150
3/3 [==============================] - 0s 870us/step - loss: 0.5000 - acc: 1.0000
Epoch 8/150
3/3 [==============================] - 0s 1ms/step - loss: 0.5000 - acc: 1.0000
Epoch 9/150
3/3 [==============================] - 0s 2ms/step - loss: 0.5000 - acc: 1.0000
Epoch 10/150
3/3 [==============================] - 0s 1ms/step - loss: 0.5000 - acc: 1.0000
Epoch 11/150
3/3 [==============================] - 0s 911us/step 

Epoch 88/150
3/3 [==============================] - 0s 940us/step - loss: 0.5000 - acc: 1.0000
Epoch 89/150
3/3 [==============================] - 0s 712us/step - loss: 0.5000 - acc: 1.0000
Epoch 90/150
3/3 [==============================] - 0s 967us/step - loss: 0.5000 - acc: 1.0000
Epoch 91/150
3/3 [==============================] - 0s 803us/step - loss: 0.5000 - acc: 1.0000
Epoch 92/150
3/3 [==============================] - 0s 782us/step - loss: 0.5000 - acc: 1.0000
Epoch 93/150
3/3 [==============================] - 0s 2ms/step - loss: 0.5000 - acc: 1.0000
Epoch 94/150
3/3 [==============================] - 0s 600us/step - loss: 0.5000 - acc: 1.0000
Epoch 95/150
3/3 [==============================] - 0s 766us/step - loss: 0.5000 - acc: 1.0000
Epoch 96/150
3/3 [==============================] - 0s 818us/step - loss: 0.5000 - acc: 1.0000
Epoch 97/150
3/3 [==============================] - 0s 2ms/step - loss: 0.5000 - acc: 1.0000
Epoch 98/150
3/3 [==============================] - 0s

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [60]:
# calculate predictions
predictions = model.predict(inputX)
pprint(predictions)

array([[0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)
